0. 필요한 라이브러리 import

In [3]:
# Basic import 라이브러리
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

# Preparing Data 라이브러리
from sklearn import preprocessing # Label Encoder 사용 위해서
from sklearn.model_selection import train_test_split

# Modeling 라이브러리
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Data Preprocessing 라이브러리
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# Scoring 라이브러리
from sklearn.metrics import f1_score

# ignore warning 라이브러리
import warnings
warnings.filterwarnings(action='ignore')

0. Basic Setting

In [8]:
class CFG:
    def __init__(self):
        self.img_size = 224
        self.epochs = 20
        self.learning_rate = 3e-4
        self.batch_size = 64
        self.seed = 41
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
cfg = CFG()
print(cfg.device)

cuda


In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(cfg.seed) # Seed 고정

In [11]:
base_dir = 'D:/Dacon_Art_Classification/Art_Classification'
data_dir = "D:/Dacon_Art_Classification/data"

1. Data Preprocessing

In [15]:
df = pd.read_csv(f'{data_dir}/train.csv')
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [16]:
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)
print(le.classes_)
df.head()

['Albrecht Du rer' 'Alfred Sisley' 'Amedeo Modigliani' 'Andrei Rublev'
 'Andy Warhol' 'Camille Pissarro' 'Caravaggio' 'Claude Monet'
 'Diego Rivera' 'Diego Velazquez' 'Edgar Degas' 'Edouard Manet'
 'Edvard Munch' 'El Greco' 'Eugene Delacroix' 'Francisco Goya'
 'Frida Kahlo' 'Georges Seurat' 'Giotto di Bondone' 'Gustav Klimt'
 'Gustave Courbet' 'Henri Matisse' 'Henri Rousseau'
 'Henri de Toulouse-Lautrec' 'Hieronymus Bosch' 'Jackson Pollock'
 'Jan van Eyck' 'Joan Miro' 'Kazimir Malevich' 'Leonardo da Vinci'
 'Marc Chagall' 'Michelangelo' 'Mikhail Vrubel' 'Pablo Picasso'
 'Paul Cezanne' 'Paul Gauguin' 'Paul Klee' 'Peter Paul Rubens'
 'Pierre-Auguste Renoir' 'Piet Mondrian' 'Pieter Bruegel' 'Raphael'
 'Rembrandt' 'Rene Magritte' 'Salvador Dali' 'Sandro Botticelli' 'Titian'
 'Vasiliy Kandinskiy' 'Vincent van Gogh' 'William Turner']


,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [21]:
train_df, val_df, _, _ = train_test_split(df, df['artist'].values, test_size=0.2, random_state=cfg.seed)

In [22]:
train_df = train_df.sort_values(by=['id'])
train_df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
5,5,./train/0005.jpg,38
6,6,./train/0006.jpg,43


In [23]:
valid_df = val_df.sort_values(by=['id'])
valid_df.head()

,id,img_path,artist
1,1,./train/0001.jpg,48
4,4,./train/0004.jpg,24
17,17,./train/0017.jpg,10
21,21,./train/0021.jpg,29
29,29,./train/0029.jpg,28
